In [1]:
suppressPackageStartupMessages(library(dplyr))
suppressPackageStartupMessages(library(tidyr))
suppressPackageStartupMessages(library(tibble))
suppressPackageStartupMessages(library(stringr))
library(parallel)
library(plyr)
library(data.table)
library(ggplot2)
library(pheatmap)
library(RColorBrewer)
library(viridis)
source('/home//jupyter/BRI_Figures_Final_V2/helper_function/helper_function_IHA.r')

------------------------------------------------------------------------------

You have loaded plyr after dplyr - this is likely to cause problems.
If you need functions from both plyr and dplyr, please load plyr first, then dplyr:
library(plyr); library(dplyr)

------------------------------------------------------------------------------


Attaching package: ‘plyr’


The following objects are masked from ‘package:dplyr’:

    arrange, count, desc, failwith, id, mutate, rename, summarise,
    summarize



Attaching package: ‘data.table’


The following objects are masked from ‘package:dplyr’:

    between, first, last


Loading required package: viridisLite



In [2]:
meta_data=read.csv("/home/jupyter/BRI_Figures_Final_V2/Dataset/SF4_meta_data-2024-05-05.csv")
meta_data$ageAtEnrollment[meta_data$ageAtEnrollment == '89+'] <- '90'
meta_data$ageAtEnrollment<-as.numeric(meta_data$ageAtEnrollment)
meta_data<-meta_data[!is.na(meta_data$ageAtEnrollment),]
meta_data$Ages<-meta_data$ageAtEnrollment
meta_data <- meta_data[!duplicated(meta_data$sample.sampleKitGuid, fromLast = TRUE), ]
meta_data$CMV<-meta_data$lab.revisionHistory.dataHistory.CMV.Ab.Screen.Result..CMVRES..newValue

In [3]:
file_list<-paste0("/home/jupyter/BRI_Figures_Final_V2/Dataset/scRNA/SF4/Average_LogNormalized_Expression/Average_LogNormalized_Expression_of_Celltypes_by_Sample_AIFI_L3/",meta_data$pbmc_sample_id,".csv")

df_list<-read_pseudobulk_expression(file_list)


[1] "Total reading time: 11.367 seconds"
[1] "The length of the list matches the length of the input path."


In [4]:
df<-read.csv("/home//jupyter/BRI_Figures_Final_V2/Figure2/02_DEG/Deseq2_Result_Y1D0.csv")

In [5]:
df<-df %>% filter(contrast=='cohort.cohortGuid')

In [6]:
df_subset<-df %>% filter(abs(log2FoldChange)>0.1,padj<0.05)

In [7]:
selected_celltype<-read.csv("/home//jupyter/BRI_Figures_Final_V2/Figure2/04_CompositeScore/BRI_CompositeScore_Down.csv")

In [8]:
keep_celltypes <- unique(selected_celltype$celltype)


In [9]:
df_subset_filtered<-df_subset%>% filter(celltype %in% keep_celltypes)


In [12]:
meta_data<-meta_data %>% arrange(Ages)
meta_data$pbmc_sample_id_gsubbed<-gsub("-",'.',meta_data$pbmc_sample_id) 
rownames(meta_data)<-meta_data$pbmc_sample_id_gsubbed

In [10]:
Up_Genes<-c('SESN3','AHNAK','PTGER2')

In [27]:
exp_all<-list()
for (gene_single in Up_Genes) {
  celltypes <- unique(df_subset_filtered %>% filter(gene == gene_single) %>%arrange(celltype)%>% select(celltype) %>% pull())
  exp_single_gene_all_celtype <- data.frame()
  for (celltype_single in celltypes) {
    exp <- lapply(df_list, function(df) {
      df <- df[gene_single, grep(celltype_single, names(df), fixed = TRUE), drop = FALSE]
    })

    exp_matrix <- do.call(cbind, exp)
    colnames(exp_matrix) <- sub(":.*", "", colnames(exp_matrix))
    
    # Convert to data frame and retain row names as a new column
    exp_df <- data.frame(exp_matrix)
    exp_df<-as.data.frame(t(exp_df))
    colnames(exp_df)<-"mean_exp"
    
    exp_df$pbmc_sample_id_gsubbed<-rownames(exp_df)
    exp_df<-left_join(exp_df,meta_data)
    
    
    
    exp_df<-exp_df %>% group_by(Ages) %>% dplyr::summarise(mean_exp=mean(mean_exp) )
    
    exp_df<-as.data.frame(exp_df)
    
    rownames(exp_df)<-exp_df$Ages
    
    exp_df<-as.data.frame(t(exp_df[2])  )
    rownames(exp_matrix)<-gene_single
    exp_df$rownames <- paste0(rownames(exp_matrix), ":", celltype_single)

    # Reorder to have rownames first (optional)
    exp_df <- exp_df[, c("rownames", setdiff(names(exp_df), "rownames"))]

    # Bind the data while keeping row names as a column
    exp_single_gene_all_celtype <- rbind.fill(exp_single_gene_all_celtype, exp_df)
  }
  
  # Optionally set the row names back if needed
rownames(exp_single_gene_all_celtype) <- exp_single_gene_all_celtype$rownames
exp_single_gene_all_celtype <- exp_single_gene_all_celtype[, -1]  # Remove the 'rownames' column after setting row names

exp_all[[gene_single]]<-exp_single_gene_all_celtype
}


Joining with `by = join_by(pbmc_sample_id_gsubbed)`
Joining with `by = join_by(pbmc_sample_id_gsubbed)`
Joining with `by = join_by(pbmc_sample_id_gsubbed)`
Joining with `by = join_by(pbmc_sample_id_gsubbed)`
Joining with `by = join_by(pbmc_sample_id_gsubbed)`
Joining with `by = join_by(pbmc_sample_id_gsubbed)`
Joining with `by = join_by(pbmc_sample_id_gsubbed)`
Joining with `by = join_by(pbmc_sample_id_gsubbed)`
Joining with `by = join_by(pbmc_sample_id_gsubbed)`
Joining with `by = join_by(pbmc_sample_id_gsubbed)`
Joining with `by = join_by(pbmc_sample_id_gsubbed)`
Joining with `by = join_by(pbmc_sample_id_gsubbed)`


In [28]:
df<-do.call(rbind,exp_all)

In [29]:
write.csv(df,"Up_Selected_Genes.csv")

In [30]:
Down_Genes<-c('IL16','STK17A')

In [31]:
exp_all<-list()
for (gene_single in Down_Genes) {
  celltypes <- unique(df_subset_filtered %>% filter(gene == gene_single) %>%arrange(celltype)%>% select(celltype) %>% pull())
  exp_single_gene_all_celtype <- data.frame()
  for (celltype_single in celltypes) {
    exp <- lapply(df_list, function(df) {
      df <- df[gene_single, grep(celltype_single, names(df), fixed = TRUE), drop = FALSE]
    })

    exp_matrix <- do.call(cbind, exp)
    colnames(exp_matrix) <- sub(":.*", "", colnames(exp_matrix))
    
    # Convert to data frame and retain row names as a new column
    exp_df <- data.frame(exp_matrix)
    exp_df<-as.data.frame(t(exp_df))
    colnames(exp_df)<-"mean_exp"
    
    exp_df$pbmc_sample_id_gsubbed<-rownames(exp_df)
    exp_df<-left_join(exp_df,meta_data)
    
    
    
    exp_df<-exp_df %>% group_by(Ages) %>% dplyr::summarise(mean_exp=mean(mean_exp) )
    
    exp_df<-as.data.frame(exp_df)
    
    rownames(exp_df)<-exp_df$Ages
    
    exp_df<-as.data.frame(t(exp_df[2])  )
    rownames(exp_matrix)<-gene_single
    exp_df$rownames <- paste0(rownames(exp_matrix), ":", celltype_single)

    # Reorder to have rownames first (optional)
    exp_df <- exp_df[, c("rownames", setdiff(names(exp_df), "rownames"))]

    # Bind the data while keeping row names as a column
    exp_single_gene_all_celtype <- rbind.fill(exp_single_gene_all_celtype, exp_df)
  }
  
  # Optionally set the row names back if needed
rownames(exp_single_gene_all_celtype) <- exp_single_gene_all_celtype$rownames
exp_single_gene_all_celtype <- exp_single_gene_all_celtype[, -1]  # Remove the 'rownames' column after setting row names

exp_all[[gene_single]]<-exp_single_gene_all_celtype
}


Joining with `by = join_by(pbmc_sample_id_gsubbed)`
Joining with `by = join_by(pbmc_sample_id_gsubbed)`
Joining with `by = join_by(pbmc_sample_id_gsubbed)`
Joining with `by = join_by(pbmc_sample_id_gsubbed)`
Joining with `by = join_by(pbmc_sample_id_gsubbed)`
Joining with `by = join_by(pbmc_sample_id_gsubbed)`
Joining with `by = join_by(pbmc_sample_id_gsubbed)`
Joining with `by = join_by(pbmc_sample_id_gsubbed)`
Joining with `by = join_by(pbmc_sample_id_gsubbed)`
Joining with `by = join_by(pbmc_sample_id_gsubbed)`
Joining with `by = join_by(pbmc_sample_id_gsubbed)`
Joining with `by = join_by(pbmc_sample_id_gsubbed)`
Joining with `by = join_by(pbmc_sample_id_gsubbed)`


In [25]:
df<-do.call(rbind,exp_all)

In [26]:
write.csv(df,"Down_Selected_Genes.csv")